In [13]:
#Code to help get all csv files into a useable format for making into figures
import pandas as pd
import numpy as np
import sys
import os
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
#directory for all regular analysis
outDir = thisDir + 'output/results/'
taskList=['semantic','mem','motor','glass']
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
#all permutations of features
subDir=outDir+'subNetwork'
randDir=outDir+'rdmNetwork'
netList=['unassign','default', 'visual','fp','dan','van','salience','co','sm','sm-lat', 'auditory','pmn','pon']
feature=[14808,10824,8736, 4620,5264,3151, 484, 4060, 2375,316, 564,45,21]
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 484),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])


#Random sampling and feature selection calculate z scores
#subnetworks overlapped with random network sampling

rdmNetSS=pd.read_csv(randDir+'/SS/nullNet_acc.csv')
rdmNetSS.drop(columns='same_sub',inplace=True)
rdmNetDS=pd.read_csv(randDir+'/DS/nullNet_acc.csv')

rdmNetBS=pd.read_csv(randDir+'/BS/nullNet_acc.csv')
rdmNetBS.drop(columns='same_sub',inplace=True)
#rdmNetCV=pd.read_csv(randDir+'/CV/nullNet_acc.csv')
rdmNetAll=pd.read_csv(randDir+'/ALL/nullNet_acc.csv')


NetSS=pd.read_csv(subDir+'/SS/acc.csv')
NetSS.drop(columns='same_sub',inplace=True)
NetSS.rename(columns={'feature':'Network','Network':'feature'},inplace=True)
NetDS=pd.read_csv(subDir+'/DS/acc.csv')
NetDS.rename(columns={'feature':'Network','Network':'feature'},inplace=True)
NetBS=pd.read_csv(subDir+'/BS/acc.csv')
NetBS.drop(columns='same_sub',inplace=True)
#NetCV=pd.read_csv(subDir+'/CV/acc.csv')
NetAll=pd.read_csv(subDir+'/ALL/acc.csv')


In [49]:
SS=NetSS.groupby(['feature','Network']).mean()
SS_z=rdmNetSS.groupby(['feature','Null_Network']).mean()
SS_z['std']=rdmNetSS.groupby(['feature','Null_Network']).std()
SS_z.reset_index(inplace=True)
SS_z.rename(columns={'Null_Network':'Network','diff':'mu'},inplace=True)
SS_zscores=SS.merge(SS_z,how='left',on=['Network'])
SS_zscores['z_score']=(SS_zscores['diff']-SS_zscores['mu'])/SS_zscores['std']

In [51]:
BS=NetBS.groupby(['feature','Network']).mean()
BS_z=rdmNetBS.groupby(['feature','Null_Network']).mean()
BS_z['std']=rdmNetBS.groupby(['feature','Null_Network']).std()
BS_z.reset_index(inplace=True)
BS_z.rename(columns={'Null_Network':'Network','diff_sub':'mu'},inplace=True)
BS_zscores=BS.merge(BS_z,how='left',on=['Network'])
BS_zscores['z_score']=(BS_zscores['diff_sub']-BS_zscores['mu'])/BS_zscores['std']

In [59]:
NetCV=NetDS.drop(columns=['diff_sub'])
rdmNetCV=rdmNetDS.drop(columns=['diff_sub'])
CV=NetCV.groupby(['feature','Network']).mean()
CV_z_mu=rdmNetCV.groupby(['feature','Null_Network']).mean()
CV_z_std=rdmNetCV.groupby(['feature','Null_Network']).std()
CV_z_mu.reset_index(inplace=True)
CV_z_std.reset_index(inplace=True)
CV_z_mu.rename(columns={'Null_Network':'Network','same_sub':'mu'},inplace=True)
CV_z_std.rename(columns={'Null_Network':'Network','same_sub':'std'},inplace=True)
CV_zscores=CV.merge(CV_z_mu,how='left',on=['Network'])
CV_zscoresFinal=CV_zscores.merge(CV_z_std,how='left',on=['Network'])
CV_zscoresFinal['z_score']=(CV_zscoresFinal['same_sub']-CV_zscoresFinal['mu'])/CV_zscoresFinal['std']

In [61]:
NetDS.drop(columns=['same_sub'],inplace=True)
rdmNetDS.drop(columns=['same_sub'],inplace=True)
DS=NetDS.groupby(['feature','Network']).mean()
DS_z_mu=rdmNetDS.groupby(['feature','Null_Network']).mean()
DS_z_std=rdmNetDS.groupby(['feature','Null_Network']).std()
DS_z_mu.reset_index(inplace=True)
DS_z_std.reset_index(inplace=True)
DS_z_mu.rename(columns={'Null_Network':'Network','diff_sub':'mu'},inplace=True)
DS_z_std.rename(columns={'Null_Network':'Network','diff_sub':'std'},inplace=True)
DS_zscores=DS.merge(DS_z_mu,how='left',on=['Network'])
DS_zscoresFinal=DS_zscores.merge(DS_z_std,how='left',on=['Network'])
DS_zscoresFinal['z_score']=(DS_zscoresFinal['diff_sub']-DS_zscoresFinal['mu'])/DS_zscoresFinal['std']

In [62]:
All=NetAll.groupby(['feature','Network']).mean()
All_z=rdmNetAll.groupby(['feature','Null_Network']).mean()
All_std=rdmNetAll.groupby(['feature','Null_Network']).std()
All_z.rename(columns={'same_sub':'cv_mu','diff_sub':'ds_mu'},inplace=True)
All_std.rename(columns={'same_sub':'cv_std','diff_sub':'ds_std'},inplace=True)
All_z=All_z.merge(All_std, on=['feature','Null_Network'])
All_z.reset_index(inplace=True)
All_z.rename(columns={'Null_Network':'Network'},inplace=True)
All_zscores=All.merge(All_z,how='left',on=['Network'])
All_zscores['CV_z_score']=(All_zscores['same_sub']-All_zscores['cv_mu'])/All_zscores['cv_std']
All_zscores['DS_z_score']=(All_zscores['diff_sub']-All_zscores['ds_mu'])/All_zscores['ds_std']

In [64]:
CV_zscoresFinal.to_csv(randDir+'/CV/zscore_all.csv',index=False)
SS_zscores.to_csv(randDir+'/SS/zscore_all.csv',index=False)
DS_zscoresFinal.to_csv(randDir+'/DS/zscore_all.csv',index=False)
BS_zscores.to_csv(randDir+'/BS/zscore_all.csv',index=False)
All_zscores.to_csv(randDir+'/ALL/zscore_all.csv',index=False)




In [3]:
rdmNetCV

,sub,task,acc,features,Null_Network
0,MSC01,semantic,0.95,14808,unassign
1,MSC01,glass,0.90,14808,unassign
2,MSC01,motor,1.00,14808,unassign
3,MSC01,mem,1.00,14808,unassign
4,MSC02,semantic,0.90,14808,unassign
...,...,...,...,...,...
5115,MSC07,mem,1.00,1640,default_co
5116,MSC10,semantic,0.80,1640,default_co
5117,MSC10,glass,0.80,1640,default_co
5118,MSC10,motor,0.95,1640,default_co


In [4]:
CV=NetCV.groupby(['feature','network','task']).mean()
CV_z=rdmNetCV.groupby(['features','Null_Network','task']).mean()
CV_z['std']=rdmNetCV.groupby(['features','Null_Network','task']).std()
CV_z.reset_index(inplace=True)
CV_z.rename(columns={'features':'feature','Null_Network':'network','acc':'mu'},inplace=True)
CV_zscores=CV.merge(CV_z,how='left',on=['network','task'])
CV_zscores['z_score']=(CV_zscores['acc']-CV_zscores['mu'])/CV_zscores['std']

In [6]:
CV_zscores.to_csv(randDir+'/CV/zscore_splitTask.csv',index=False)

In [17]:

DS=NetDS.groupby(['task','feature','network']).mean()
DS_z=rdmNetDS.groupby(['task','features','Null_Network']).mean()
DS_z['std']=rdmNetDS.groupby(['task','features','Null_Network']).std()
DS_z.reset_index(inplace=True)
DS_z.rename(columns={'features':'feature','Null_Network':'network','acc':'mu'},inplace=True)
DS_zscores=DS.merge(DS_z,how='left',on=['network','task'])
DS_zscores['z_score']=(DS_zscores['acc']-DS_zscores['mu'])/DS_zscores['std']

In [19]:
DS_zscores.to_csv(randDir+'/DS/zscore_splitTask.csv',index=False)

In [29]:
NetCV=NetDS.drop(columns=['diff_sub'])
rdmNetCV=rdmNetDS.drop(columns=['diff_sub'])
CV=NetCV.groupby(['feature','Network','task','train']).mean()
CV_z=rdmNetCV.groupby(['feature','Null_Network','task','train']).mean()
CV_z['std']=rdmNetCV.groupby(['feature','Null_Network','task','train']).std()
CV_z.reset_index(inplace=True)
CV_z.rename(columns={'Null_Network':'Network','same_sub':'mu'},inplace=True)
CV_zscores=CV.merge(CV_z,how='left',on=['Network','task','train'])
CV_zscores['z_score']=(CV_zscores['same_sub']-CV_zscores['mu'])/CV_zscores['std']
CV_zscores.replace([np.inf,-np.inf],0,inplace=True)
CV_zscores.replace([np.nan],0,inplace=True)

In [36]:
CV_zscores.to_csv(randDir+'/CV/zscore_splitSub.csv',index=False)

In [34]:
#NetDS.drop(columns=['same_sub'],inplace=True)
#rdmNetDS.drop(columns=['same_sub'],inplace=True)
DS=NetDS.groupby(['task','feature','Network','train']).mean()
DS_z=rdmNetDS.groupby(['task','feature','Null_Network','train']).mean()
DS_z['std']=rdmNetDS.groupby(['task','feature','Null_Network','train']).std()
DS_z.reset_index(inplace=True)
DS_z.rename(columns={'Null_Network':'Network','diff_sub':'mu'},inplace=True)
DS_zscores=DS.merge(DS_z,how='left',on=['Network','task','train'])
DS_zscores['z_score']=(DS_zscores['diff_sub']-DS_zscores['mu'])/DS_zscores['std']
DS_zscores.replace([np.inf,-np.inf],0,inplace=True)
DS_zscores.replace([np.nan],0,inplace=True)

In [37]:
DS_zscores.to_csv(randDir+'/DS/zscore_splitSub.csv',index=False)

In [39]:
DS_zscores

,Network,task,train,diff_sub,feature,mu,std,z_score
0,pon,glass,MSC01,0.615827,21,0.557547,0.052535,1.109367
1,pon,glass,MSC02,0.498561,21,0.561863,0.049920,-1.268065
2,pon,glass,MSC03,0.579137,21,0.556978,0.050851,0.435750
3,pon,glass,MSC04,0.602878,21,0.559209,0.053245,0.820156
4,pon,glass,MSC05,0.542446,21,0.559014,0.049561,-0.334305
5,pon,glass,MSC06,0.647619,21,0.569857,0.052898,1.470038
6,pon,glass,MSC07,0.633094,21,0.559662,0.042692,1.720013
7,pon,glass,MSC10,0.582014,21,0.555353,0.057896,0.460515
8,pmn,glass,MSC01,0.494245,45,0.570000,0.051118,-1.481971
9,pmn,glass,MSC02,0.597122,45,0.579245,0.046225,0.386758
